In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np

import sys
import os

sys.path.append(os.environ['REPO_DIR'] + '/utilities')
from utilities2015 import *
from data_manager import *
from metadata import *

from conversion import images_to_volume

import matplotlib.pyplot as plt
%matplotlib inline

import time

Setting environment for Gordon


No vtk


In [2]:
# for stack in ['MD589', 'MD594', 'MD585']:
# for stack in all_stacks:
#     if stack in ['MD589', 'MD594', 'MD585', 'MD592']:
#         continue
for stack in ['MD635']:

    first_sec, last_sec = metadata_cache['section_limits'][stack]

    bar = show_progress_bar(first_sec, last_sec)

    images = {}
    for sec in range(first_sec, last_sec):

        bar.value = sec

        try:
            fn = DataManager.get_image_filepath(stack, section=sec, resol='thumbnail', version='cropped_tif')
            img = img_as_ubyte(rgb2gray(imread(fn)))

            # apply foreground mask
            mask = DataManager.load_thumbnail_mask_v2(stack=stack, section=sec, version='aligned_cropped')
            img[~mask] = 0

            images[sec-1] = img
        except:
            pass

    downsample = 32
    voxel_z_size = section_thickness/(xy_pixel_distance_lossless * downsample)
    volume, volume_bbox = images_to_volume(images=images, voxel_size=(1, 1, voxel_z_size), first_sec=first_sec-1, last_sec=last_sec-1)

    output_dir = create_if_not_exists(os.path.join(VOLUME_ROOTDIR, stack))

    bp.pack_ndarray_file(volume, os.path.join(output_dir, '%(stack)s_down%(downsample)dVolume.bp') % \
                         dict(stack=stack, downsample=downsample))

    np.savetxt(os.path.join(output_dir, '%(stack)s_down%(downsample)dVolume_bbox.txt') % \
               dict(stack=stack, downsample=downsample),
               np.array(volume_bbox, np.int)[None, :], fmt='%d')

Widget Javascript not detected.  It may not be installed properly. Did you enable the widgetsnbextension? If not, then run "jupyter nbextension enable --py --sys-prefix widgetsnbextension"
/oasis/projects/nsf/csd395/yuncong/brain_virtualenv/lib/python2.7/site-packages/skimage/util/dtype.py:110: UserWarning: Possible precision loss when converting from float64 to uint8
  "%s to %s" % (dtypeobj_in, dtypeobj))
/oasis/projects/nsf/csd395/yuncong/brain_virtualenv/lib/python2.7/site-packages/skimage/external/tifffile/tifffile.py:1794: RuntimeWarning: py_decodelzw encountered unexpected end of stream
  strip = decompress(strip)
